To begin upload this notebook to your own drive:

### Submission Instructions:
1. **Restart the kernel** (in the menubar, select Runtime$\rightarrow$Restart runtime)
2. **Run all cells** (in the menubar, select Runtime$\rightarrow$Run All).
3. **Download the notebook** (in the menubar, select File$\rightarrow$Download .ipynb)
4. **Upload the downloaded notebook (.ipynb file) to your repository**.

Note: To use a GPU, do the following: Runtime$\rightarrow$Change runtime type$\rightarrow$ GPU

Make sure you fill in any place that says `YOUR CODE HERE`, and that no tests fail.  

# ![shakespeare](https://i.imgur.com/81YZuel.jpg)

# Generating Shakespeare Using a Character-level Language Model

### From Words to Characters
In the previous section we dealt with word-level language models. But looking again at section 1, there is nothing that constraints us to using _words_ as the basic elemnents in our model. The model we analyzed in section 1 could just as well be character-based - just replace "word" with "character", and you are good to go. In this notebook we will train a small character-based language model that will help us generate Shakespearean-like (emphasis on the _like_...) texts.

### Question 2.a
Can you think of an advantage a character-based language model could have over a word-based language model? And what about the other way around: can you think of an advantage a word-based language model could have over a character-based language model? (Add your answer to the final submission pdf).

### Using PyTorch

We'll build our language model using PyTorch. PyTorch is a [very popular](https://thegradient.pub/state-of-ml-frameworks-2019-pytorch-dominates-research-tensorflow-dominates-industry/) open-source machine learning (and deep learning) framework developed by Facebook. In short:

> Pytorch is a Python-based scientific computing package targeted at two sets of audiences:
* A replacement for NumPy to use the power of GPUs
* A deep learning research platform that provides maximum flexibility and speed

To get familiar with PyTorch, check out this [quick tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html). In addition, another imporant difference from numpy is that PyTorch can automatically calculate the gradients needed for backpropagation, as explained [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html).

### Preparing the Data

Our dataset is a plain text file. For simplicity, we turn any potential unicode characters into plain ASCII by using the `unidecode` package.

In [1]:
!pip install unidecode

In [2]:
import string
import random
# import re
import requests
import unidecode
url = "https://github.com/tau-nlp-course/NLP_HW2/raw/main/data/shakespeare.txt"


all_characters = string.printable
n_characters = len(all_characters)  # our vocabulary size (|V| from the handout)

dataset_as_string = unidecode.unidecode(requests.get(url).content.decode())
n_chars_in_dataset = len(dataset_as_string)
print(f'Total number of characters in our dataset: {n_chars_in_dataset}')

Total number of characters in our dataset: 1115394


To make inputs out of this big string of text, we will split it into chunks.

In [3]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, n_chars_in_dataset - chunk_len)
    end_index = start_index + chunk_len + 1
    return dataset_as_string[start_index:end_index]

print(random_chunk())

d.

BUCKINGHAM:
My Lord!

KING RICHARD III:
Ay, what's o'clock?

BUCKINGHAM:
I am thus bold to put your grace in mind
Of what you promised me.

KING RICHARD III:
Well, but what's o'clock?

BUCKINGHAM:
Upon the stroke of ten.

KING RICHARD III:
Well, let it strike.

BUCKINGHAM:
Why let it strike?

KING RICHARD III:
Because that, like a Jack, thou keep'st the stroke
Betwixt thy begging and my meditat


### Building Our Model

Our model consists of three main components:

1. [**Embedding**](https://pytorch.org/docs/stable/nn.html#embedding). A mapping between characters and their learned representations ("word vectors") \[correspoding to ${\boldsymbol L}$ in terms of the handout\]
2. [**GRU**](https://pytorch.org/docs/stable/nn.html#gru). \[correspoding to the computation of ${\boldsymbol h}^{(t)}$ in terms of the handout\]
3. **Output Layer**. A feed-forward neural network that transforms a hidden state at a timestep into a probability distribution of the next character. \[correspoding to the computation of $\hat{\boldsymbol y}^{(t)}$ in terms of the handout\]

### Question 2.b
Complete the implementation of the `forward` method of our model.

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class OurModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(OurModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(num_embeddings=input_size,
                                      embedding_dim=hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, input_, hidden):
        # General instructions:
        # Pass the embedded input through the GRU and use the output layer to get the next character distribution.
        # return that distribution and the next hidden state.
        # You may need to play around with the dimensions a bit until you get it right. Dimension-induced frustration is good for you!
        # -------------------------
        # YOUR CODE HERE
        if isinstance(input_, int):  # Python int
            input_ = torch.tensor([[input_]], dtype=torch.long, device=next(self.parameters()).device)
        elif isinstance(input_, torch.Tensor):
            if input_.dim() == 0:
                input_ = input_.view(1, 1)
            elif input_.dim() == 1:
                input_ = input_.view(1, -1)

        embedded  = self.embedding(input_) # (1, batch, hidden_size)
        output, hidden = self.gru(embedded, hidden) # (1, batch, hidden_size)
        output = self.output_layer(output.squeeze(0)) # (batch, output_size)
        # -------------------------
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.num_layers, 1, self.hidden_size))

### Creating the Training Examples

Each chunk will be turned into a tensor by looping through the characters of the string and looking up the index of each character in `all_characters`.

In [5]:
# Turn a string into list of longs
def chars_to_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(chars_to_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


Now we can assemble a pair of input and target tensors (i.e. a single training example) for training, from a random chunk. The input will be all characters *except the last*, and the target will be all characters *except the first*. So if our chunk is "abc" the input will correspond to "ab" while the target is "bc".

In [6]:
def random_training_set():
    chunk = random_chunk()
    inp = chars_to_tensor(chunk[:-1])
    target = chars_to_tensor(chunk[1:])
    return inp, target

### Evaluating

To evaluate the network we will feed one character at a time, use the outputs of the network as a probability distribution for the next character, and repeat. To start generation we pass a priming string to start building up the hidden state, from which we then generate one character at a time.

In [7]:
import torch.nn.functional as F

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = model.init_hidden()
    prime_input = chars_to_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]

    for p in range(predict_len):
        output, hidden = model(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist =  F.softmax(output / temperature, dim=-1)
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = chars_to_tensor(predicted_char)

    return predicted

### Training

The main training function

In [8]:
def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = model(inp[c], hidden)
        loss += criterion(output, target[c].view(-1))

    loss.backward()
    optimizer.step()

    return loss.item() / chunk_len

A helper to print the amount of time passed:

In [9]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return f'{m}m {math.floor(s)}s'

In [10]:
# DO NOT DELETE THIS CELL


Define the training parameters, instantiate the model, and start training:

In [11]:
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100  # (D_h from the handout)
num_layers = 1
lr = 0.005

model = OurModel(n_characters, hidden_size, n_characters, num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set())
    loss_avg += loss

    if epoch % print_every == 0:
        print(f'[time elapsed: {time_since(start)}  ;  epochs: {epoch} ({epoch / n_epochs * 100}%)  ;  loss: {loss:.4}]')
        print(evaluate('Wh', 200), '\n')  # generate text starting with 'Wh'

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

import numpy as np
np.save('all_losses.npy', np.array(all_losses))

torch.save(
    {
        "state_dict": model.state_dict(),
        "stoi": {ch: i for i, ch in enumerate(all_characters)}
    },
    "models/char_rnn_model.pth"
)
print("Char-RNN weights & vocab saved to models/char_rnn_model.pth")

[time elapsed: 0m 23s  ;  epochs: 100 (5.0%)  ;  loss: 2.229]
Whe sherd asa asst is ma thes that as say hate lol ow sis at doldls hin, withy bond ids ford be thy his breat loust:
Whalt salle, ar pulll apllt nont fis all lomy rerat?

RKENGLOL:
Hom chat he,
Whe ald  

[time elapsed: 0m 44s  ;  epochs: 200 (10.0%)  ;  loss: 1.983]
Who wircem;
Or the, his hi the 'gath the his haal shat rand, for to tell in: mar preang in greter May,
On in wit mey that be frion thal'd thing. Gow me reteve shat, fon, were non hathe fors' and and pro 

[time elapsed: 1m 5s  ;  epochs: 300 (15.0%)  ;  loss: 2.01]
Whe no'r to the ther swalours stich an there lilsends to here hooles.

PROUCHETRO:
It my mory wit'-genstore coull trowed to my this hand to o; the he blair.

HPRICKI:
It the mare to mad you, decorlmore
 

[time elapsed: 1m 26s  ;  epochs: 400 (20.0%)  ;  loss: 2.072]
Whersence for it forser an all to shands then dour aal
He fave pist this beot be-can mes,
And the and reap chars thest death,
Nour crand

KeyboardInterrupt: 

### Training Loss

Plotting the the losses that were computed during training can provide a further indication that the network was indeed learning (Add your plot to the final submission pdf).

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
#
# plt.plot(all_losses)
# plt.grid(True)
# plt.xlabel('# of epochs (divided by plot_every)')
# plt.ylabel('average loss')
# plt.show()

In [ ]:
"""
The cell you provided crashes my kernel constantly.
I decided to save the loss list and plot it myself.
"""

import numpy as np
import matplotlib.pyplot as plt

all_losses = np.load('all_losses.npy')

plt.plot(all_losses)
plt.grid(True)
plt.xlabel('# of epochs (divided by plot_every)')
plt.ylabel('average loss')
plt.tight_layout()
plt.show()

